In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests, datetime, talib, json, pprint, websocket
from binance.client import Client
from tradingview_ta import TA_Handler, Interval

In [ ]:
api_key = 'wZGVBCNG0qpwEA0wmg6Di2CkA28zkpqFn0zf0jk8lBXJ7eM9R1qA6e8Ike1dP9ka'
api_secret = '56sXANiThjsRM3WesAXr2kyoo45omvUqeAjdl3f9dKQdbyVlzwrThXXoKRDmPG2k'
client = Client(api_key, api_secret)

In [ ]:
klines = client.get_historical_klines('BTCUSDT', client.KLINE_INTERVAL_1MINUTE, "10 minutes ago UTC")
df = pd.DataFrame(klines)
df = df.iloc[:,:5]
df.columns = ['date','open','high','low','close']
df['date'] = pd.to_datetime(df['date'], unit = 'ms')
df['RSI6'] = 'Nan'
df.iloc[:,1:5] = df.iloc[:,1:5].astype(float)
df

In [ ]:
link = 'wss://stream.binance.com:9443/ws/btcusdt@kline_1m'
closes =[]

In [ ]:
def clean_data(data):
    df = pd.DataFrame(data)
    df = df.transpose()
    df.columns = ['date','open','high','low','close','RSI6']
    df['date'] = pd.to_datetime(df['date'], unit = 'ms')
    df.iloc[:,1:5] = df.iloc[:,1:5].astype(float)
    return df

In [ ]:
def on_close(ws):
    print('closed connection')
def on_open(ws):
    print('opened connection')

def on_message(ws, ms):

    global closes

    js_message = json.loads(ms)
    candle = js_message['k']
    is_candle_close = candle['x']
    close = candle['c']

    if is_candle_close:
        row = []
        row.append(candle['t'])
        row.append(candle['o'])
        row.append(candle['h'])
        row.append(candle['l'])
        row.append(candle['c'])
        row.append('Nan')
        print(row)
        df.loc[len(df)] = row
        df.iloc[-1:] = clean_data(df.loc[len(df)-1])

        np_closes = np.array(df['close'])

        df['RSI6'] = talib.RSI(np_closes.astype(float), 6)
        print(df)
   
        

    
ws = websocket.WebSocketApp(link,on_open=on_open, on_close = on_close, on_message=on_message)
ws.run_forever()

In [ ]:
np_closes = np.array(df['close'])

np_closes.astype(float)

df['RSI6'] = talib.RSI(np_closes.astype(float),6)

In [10]:
output = TA_Handler(screener = 'Crypto', exchange = 'Binance', symbol= 'BTCUSDTPERP', interval= Interval.INTERVAL_5_MINUTES)
output.get_analysis().indicators['BB.lower']

KeyboardInterrupt: 

In [13]:
import talib

closes = [23779.47,23762.54,23755.12,23751.95,23747.91,23769.31,23751.42,23746.65,23729.27,23748.46]
np_closes = np.array(closes)

close_delta = np_closes.diff()
up = close_delta.clip(lower=0)
down = -1 * close_delta.clip(upper=0)

AvgU = up.rolling(window = 6).mean()
AvgD = down.rolling(window = 6).mean()

rs = AvgU / AvgD
rsi_simple = 100 - (100/(1 + rs))
rsi_simple_np = np.array(rsi_simple)
rsi_simple_np

AttributeError: 'numpy.ndarray' object has no attribute 'diff'